In [3]:
from twarc import Twarc2

In [4]:
import yaml

In [5]:
with open('../bearer_token.yaml') as f:
    bearer_token = yaml.safe_load(f)['bearer_token']

In [6]:
import pandas as pd

In [18]:
tweets = pd.read_csv('../test_data/twacapic-parliamentarians-tryout-2.csv', lineterminator='\n', dtype=str)

In [20]:
ids = tweets['id']

In [77]:
test_ids = ids #.sample(123)

In [35]:
t = Twarc2(bearer_token=bearer_token)

In [36]:
from sys import stdout

In [ ]:
# %%time
from math import ceil

lookup = t.tweet_lookup(test_ids.values)

errors = list()

pages = 0

for page in lookup:
    pages += 1
    
    for tweet in page['data']:
        if 'withheld' in tweet.keys(): # check whether tweet is withheld in any country
            errors.append(tweet) # if so, append tweet object to errors
    
    page_length = len(page['data'])

    if page_length != 100: # Now focus on 'incomplete' pages

        if len(test_ids) >= pages * 100: # There should be errors if page is not last page
            assert 'errors' in page.keys()

        n = 0
        
        if 'errors' in page.keys():
            for error in page['errors']:
                if error['parameter'] == 'ids' and error['resource_type'] == 'tweet': 
                # These fields seem to mark a 'deleted' or 'protected' tweet error
                
                    assert error['value'] in test_ids.values # just to be sure that our logic is sound
                    
                    errors.append(error)
                    
                    n += 1
        
        if len(test_ids) >= pages * 100:     # If page is not last page
            assert page_length == 100 - n    # the number of errors should explain the missing tweets.

    stdout.write(f'\r{pages}')
    stdout.flush()

assert pages == ceil(len(test_ids) / 100)    # Have we processed all pages?
            

In [80]:
import ndjson
from datetime import datetime

In [81]:
with open(f'{datetime.strftime(datetime.now(), "%Y%m%d")}_deleted_tweets.ndjson', 'w') as f:  # write to file with today's date in front
    ndjson.dump(errors, f)